<a href="https://colab.research.google.com/github/massivetriceps/AI/blob/main/week5/assignment/diabetes_classification_prc2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import mean_squared_error
import torch
from torch import nn
from torch.utils.data import TensorDataset, DataLoader
import torch.optim as optim

In [ ]:
import pandas as pd
from google.colab import files

# CSV 파일 업로드
uploaded = files.upload()

# 파일 이름이 'diabetes.csv'라면
df = pd.read_csv('diabetes.csv')

# 데이터 확인
df.head()

Saving diabetes.csv to diabetes.csv


,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [ ]:
df.columns

Index(['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin',
       'BMI', 'DiabetesPedigreeFunction', 'Age', 'Outcome'],
      dtype='object')

In [ ]:
X = df.drop("Outcome", axis=1).values
y = df["Outcome"].values.astype(np.float32)

In [ ]:
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [ ]:
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.reshape(-1, 1), dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.reshape(-1, 1), dtype=torch.float32)

In [ ]:
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32)

In [ ]:
model = nn.Sequential(
    nn.Linear(8, 64),
    nn.ReLU(),
    nn.Linear(64, 32),
    nn.ReLU(),
    nn.Linear(32, 1),
    nn.Sigmoid()
)

In [ ]:
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
for epoch in range(20):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    print(f"Epoch [{epoch+1}/20], Loss: {running_loss/len(train_loader):.4f}")


Epoch [1/20], Loss: 0.7175
Epoch [2/20], Loss: 0.6452
Epoch [3/20], Loss: 0.5717
Epoch [4/20], Loss: 0.5201
Epoch [5/20], Loss: 0.4806
Epoch [6/20], Loss: 0.4543
Epoch [7/20], Loss: 0.4496
Epoch [8/20], Loss: 0.4391
Epoch [9/20], Loss: 0.4333
Epoch [10/20], Loss: 0.4286
Epoch [11/20], Loss: 0.4319
Epoch [12/20], Loss: 0.4150
Epoch [13/20], Loss: 0.4207
Epoch [14/20], Loss: 0.4224
Epoch [15/20], Loss: 0.4116
Epoch [16/20], Loss: 0.4252
Epoch [17/20], Loss: 0.4134
Epoch [18/20], Loss: 0.4162
Epoch [19/20], Loss: 0.4082
Epoch [20/20], Loss: 0.4152


In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix
model.eval()
with torch.no_grad():
    predictions = model(X_test_tensor)
    predicted_classes = (predictions > 0.5).float()
    acc = accuracy_score(y_test_tensor, predicted_classes)
    cm = confusion_matrix(y_test_tensor, predicted_classes)

print(f"Test Accuracy: {acc:.4f}")
print("Confusion Matrix:\n", cm)

Test Accuracy: 0.7727
Confusion Matrix:
 [[83 16]
 [19 36]]
